In [ ]:
from mpl_toolkits.basemap import Basemap
from multiprocessing import Process,Array,Value
from collections import defaultdict
from scipy.spatial import distance as dst
import matplotlib.pyplot as plt
import shapely.geometry as sp
import numpy as np
import pandas as pd
import pickle as pk
import datetime as dt
import threading
import itertools
import math
import time
import json
import copy

In [ ]:
flights=pd.read_csv("Outputs/cleanFlightHistory.csv",low_memory=False)
Tracks=pd.read_csv("Outputs/realTracks.csv")
m = pk.load(open("Outputs/M_ConversionMetric.pkl", "rb"))
speeds = pk.load(open("Outputs/SpeedDict.pkl", "rb"))
SectorGraph = pk.load(open("Outputs/SectorGraph.pkl", "rb"))
Centroids = pk.load(open("Outputs/CentroidDict.pkl", "rb"))
ConvexHulls = pk.load(open("Outputs/ConvexDict.pkl", "rb"))
airportSector=pk.load(open("Outputs/airportSectorDict.pkl", "rb"))

In [ ]:
flightID=

In [ ]:
speedinKnots=speeds[flightID]
flight=flights[flights['id']==flightID]
start=airportSector[flight['departure_airport_icao_code'].values[0]]
end=airportSector[flight['arrival_airport_icao_code'].values[0]]
print(f"Input to GA:\n{start},{end},0,{speedinKnots}")

realDepart=dt.datetime.fromisoformat(str(flight['actual_runway_departure'].values[0]))
realArrival=dt.datetime.fromisoformat(str(flight['actual_runway_arrival'].values[0]))
flights[flights['id']==flightID]

In [ ]:
%matplotlib qt
fig = plt.figure(pk.load(open("Outputs/Simulator.pkl","rb")))

In [ ]:
pathFromGA=[989,980,983,998,990]

In [ ]:
singleFlightTrack=Tracks[Tracks['flighthistory_id']==flightID]
singleFlightTrack=singleFlightTrack.sort_values('received')
lat=singleFlightTrack['latitude_degrees'].values
lon=singleFlightTrack['longitude_degrees'].values
realX,realY=m(lon,lat)
PathLenGA=0
PathLenReal=0
for pt in range(1,len(realX)):
     PathLenReal+=dst.euclidean((realX[pt-1],realY[pt-1]),(realX[pt],realY[pt]))
mpm_speed=(speedinKnots/1.944)*60
GAx=[Centroids[pathFromGA[0]][0]]
GAy=[Centroids[pathFromGA[0]][1]]
for Hull in range(len(pathFromGA)-1):
    X=[]
    Y=[]
    Plot=ConvexHulls[pathFromGA[Hull]]
    Plot.append(ConvexHulls[pathFromGA[Hull]][0])
    for pt in Plot:
        X.append(pt[0])
        Y.append(pt[1])
    line1 = sp.LineString(list(Plot))
    for neigh in SectorGraph.neighbors(pathFromGA[Hull]):
        if(neigh==pathFromGA[Hull+1]):
            line2 = sp.LineString([Centroids[pathFromGA[Hull]], Centroids[neigh]])
            ABC=line1.intersection(line2)
            PathLenGA+=dst.euclidean((GAx[-1],GAy[-1]),(ABC.x,ABC.y))
            GAx.append(ABC.x)
            GAy.append(ABC.y)
PathLenGA+=dst.euclidean((GAx[-1],GAy[-1]),Centroids[pathFromGA[-1]])
GAx.append(Centroids[pathFromGA[-1]][0])
GAy.append(Centroids[pathFromGA[-1]][1])

In [ ]:
plt.plot(realX,realY,linewidth=20,c='b',label="Actual Path")
plt.plot(GAx,GAy,linewidth=20,c='g',label="GA Path")
plt.legend(loc="upper left",prop={'size': 100})
fig.canvas.draw()

In [ ]:
print(PathLenGA/mpm_speed)
print(PathLenReal/mpm_speed)
T=((realArrival-realDepart).seconds//3600)*60+(((realArrival-realDepart).seconds//60)%60)
print(T)

In [ ]:
((PathLenReal/(T*60))*1.944)

In [ ]:
(PathLenReal/(T*60))

In [ ]:
import time
dfFlights=pd.read_csv("Outputs/cleanFlightHistory.csv")
ConvexDict = pk.load(open("Outputs/ConvexDict.pkl", "rb"))
dayDf=dfFlights[dfFlights['scheduled_runway_departure'].str.contains("2013-08-16")].copy()
dayDf.sort_values(by=["scheduled_runway_departure"], inplace = True)
dayDf=dayDf[:20]
stlol=time.time()
ProperPath=0
BadPath=0
NoPath=[]
HalfPath=[]
FullPath=[]
for index,flight in dayDf.iterrows():
    start=airportSector[flight['departure_airport_icao_code']]
    end=airportSector[flight['arrival_airport_icao_code']]
    if(start==end):
        continue
    singleFlightTrack=Tracks[Tracks['flighthistory_id']==flight['id']]
    singleFlightTrack=singleFlightTrack.sort_values('received')
    lat=singleFlightTrack['latitude_degrees'].values
    lon=singleFlightTrack['longitude_degrees'].values
    realX,realY=m(lon,lat)
    if(len(realX)==0):
        NoPath.append(flight['id'])
    else:
        StartPoint=(realX[0],realY[0])
        EndPoint=(realX[-1],realY[-1])
        if(not ((sp.Polygon(ConvexDict[start]).contains(sp.Point(StartPoint))) and (sp.Polygon(ConvexDict[end]).contains(sp.Point(EndPoint))))):
            HalfPath.append(flight['id'])
        else:
            FullPath.append(flight['id'])
enlol=time.time()
print(enlol-stlol)

In [ ]:
print(len(NoPath))
print(len(HalfPath))
print(len(FullPath))